In [1]:
import requests
import pandas as pd
import json
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
from load_snowflake import load_snowflake
from urllib.parse import quote_plus 
load_dotenv()

api_key = os.getenv('API_KEY')

In [2]:
def fetch_events():
    # Your Ticketmaster API key
    api_key = os.getenv('API_KEY')
    
    # Base URL for Ticketmaster API
    url = 'https://app.ticketmaster.com/discovery/v2/events'
    
    # Initial API parameters
    params = {
        'apikey': api_key,
        'countryCode': 'US',                      # Limit results to the US
        'startDateTime': '2024-12-01T00:00:00Z',  # Start date
        ### Deep paging issues over 1000 items, limiting to 1 month for now. will need to add batching
        'endDateTime': '2024-12-31T23:59:59Z',    # End date
        'size': 200,                              # Max events per page
        'page': 0,                                # Start at page 0
        'sort': 'date,asc'                        # Sort by date ascending
    }
    
    all_events = []  # To store all events data
    
    while True:
        # Fetch the events for the current page
        response = requests.get(url, params=params)
        
        # Check if the request was successful
        if response.status_code != 200:
            print(f"Error: Unable to fetch data (status code {response.status_code})")
            break
        
        # Get the JSON response
        data = response.json()

        # Check if there are any events in the response
        if '_embedded' not in data or 'events' not in data['_embedded']:
            print("No more events found.")
            break
        
        # Extract events from the current page
        events = data['_embedded']['events']
        all_events.extend(events)
        
        # Print the number of events fetched on this page
        print(f"Fetched {len(events)} events on page {params['page']}")
        
        # Check if there are more pages
        if params['page'] >= data['page']['totalPages'] - 1:
            print("All pages have been fetched.")
            break
        
        # Increment to the next page
        params['page'] += 1
    
    
    return all_events

In [3]:
def events_to_dataframe(events):
    # Create a list to store processed event data
    event_list = []

    for event in events:
        # Extract relevant fields from each event
        event_name = event.get('name')
        event_date = event['dates']['start'].get('localDate')
        event_time = event['dates']['start'].get('localTime', 'N/A')  
        venue_name = event['_embedded']['venues'][0].get('name')
    #    venue_city = event['_embedded']['venues'][0]['city'].get('name', 'N/A')
     #   venue_state = event['_embedded']['venues'][0]['state'].get('name', 'N/A')  
        
        # Append event data to the list
        event_list.append([event_name, event_date, event_time, venue_name #, venue_city, venue_state
                           ])

    # Convert the list to a DataFrame
    df = pd.DataFrame(event_list, columns=['Event Name', 'Date', 'Time', 'Venue' #, 'City', 'State'
                                           ])
    return df

In [4]:
def events_to_dataframe_allCol(events):
    df = pd.json_normalize(events, sep='_')

    return df


In [5]:
events_data = fetch_events()
events_df = events_to_dataframe_allCol(events_data)


Fetched 200 events on page 0
Fetched 200 events on page 1
Fetched 200 events on page 2
Fetched 200 events on page 3
Fetched 200 events on page 4
Error: Unable to fetch data (status code 400)


In [6]:
def init_drop(df: pd.DataFrame):

    drop_cols = [
        'images',
        'locale',
        'test'
    ]

    df = df.drop(columns=drop_cols, errors = 'ignore')

    return df

In [7]:
df_clean1 = init_drop(events_df)

In [8]:
print(df_clean1.columns)

Index(['name', 'type', 'id', 'url', 'classifications', 'info', 'pleaseNote',
       'priceRanges', 'sales_public_startDateTime', 'sales_public_startTBD',
       'sales_public_startTBA', 'sales_public_endDateTime',
       'dates_access_startDateTime', 'dates_access_startApproximate',
       'dates_access_endDateTime', 'dates_access_endApproximate',
       'dates_start_localDate', 'dates_start_localTime',
       'dates_start_dateTime', 'dates_start_dateTBD', 'dates_start_dateTBA',
       'dates_start_timeTBA', 'dates_start_noSpecificTime',
       'dates_end_localDate', 'dates_end_localTime', 'dates_end_dateTime',
       'dates_end_approximate', 'dates_end_noSpecificTime', 'dates_timezone',
       'dates_status_code', 'dates_spanMultipleDays', '_links_self_href',
       '_links_venues', '_embedded_venues', '_links_attractions',
       '_embedded_attractions', 'ticketing_safeTix_enabled',
       'ticketing_safeTix_inAppOnlyEnabled', 'ticketing_id', 'description',
       'promoters', 'promo

In [9]:
def transform_data(df: pd.DataFrame):
    
    # contains incompatible info for snowflake toSQL function, information does not seem too important besides maybe aliases. 
    # might circle back to fixing these columns at a later date. 
    exclude = [
    '_embedded_venues_markets',
    '_embedded_venues_dmas',
    '_embedded_venues_images',
    '_embedded_venues_aliases',
    '_links_attractions',
    '_embedded_attractions',
    'promoters',
    'products',
    'sales_presales',
    'outlets'
]    
    df = df.drop(columns=exclude, errors='ignore')

    return df

def flatten_nested_json(df, prefix_sep='_'):
    # Initialize an empty DataFrame to store the flattened data
    flattened_df = pd.DataFrame()

    for column in df.columns:
        # If the column has a nested dictionary or list of dictionaries, flatten it
        if isinstance(df[column].iloc[0], dict):
            # Use json_normalize to flatten the nested dictionary
            expanded_df = pd.json_normalize(df[column], sep=prefix_sep)
            expanded_df.columns = [f"{column}{prefix_sep}{subcol}" for subcol in expanded_df.columns]
            flattened_df = pd.concat([flattened_df, expanded_df], axis=1)
        elif isinstance(df[column].iloc[0], list) and isinstance(df[column].iloc[0][0], dict):
            # Flatten lists of dictionaries
            expanded_df = pd.json_normalize(df[column].explode().dropna(), sep=prefix_sep)
            expanded_df.columns = [f"{column}{prefix_sep}{subcol}" for subcol in expanded_df.columns]
            flattened_df = pd.concat([flattened_df, expanded_df], axis=1)
        else:
            # If the column is not nested, add it directly
            flattened_df[column] = df[column]

    return flattened_df

In [10]:
flat_df = flatten_nested_json(df_clean1)

In [11]:

df_clean = transform_data(flat_df)
print("#####printing clean df####")
print(df_clean.dtypes.to_string())

#####printing clean df####
name                                                     object
type                                                     object
id                                                       object
url                                                      object
classifications_primary                                  object
classifications_family                                   object
classifications_segment_id                               object
classifications_segment_name                             object
classifications_genre_id                                 object
classifications_genre_name                               object
classifications_type_id                                  object
classifications_type_name                                object
classifications_subType_id                               object
classifications_subType_name                             object
classifications_subGenre_id                              object
classificatio

In [12]:
import requests
import pandas as pd
import json
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine


def load_snowflake(df: pd.DataFrame, table_name: str):

    load_dotenv()
    # Snowflake login from .env
    SNOWFLAKE_USER = os.getenv('SNOWFLAKE_USER')
    SNOWFLAKE_PASSWORD = os.getenv('SNOWFLAKE_PASSWORD')
    SNOWFLAKE_ACCOUNT = os.getenv('SNOWFLAKE_ACCOUNT')
    SNOWFLAKE_WAREHOUSE = os.getenv('SNOWFLAKE_WAREHOUSE')
    SNOWFLAKE_DATABASE = os.getenv('SNOWFLAKE_DATABASE')
    SNOWFLAKE_SCHEMA = os.getenv('SNOWFLAKE_SCHEMA')

    engine = create_engine(f'snowflake://{SNOWFLAKE_USER}:{SNOWFLAKE_PASSWORD}@{SNOWFLAKE_ACCOUNT}/{SNOWFLAKE_DATABASE}/{SNOWFLAKE_SCHEMA}?warehouse={SNOWFLAKE_WAREHOUSE}')

    try:
        df.to_sql(table_name, con=engine, index=False, if_exists='replace')
        num_events = len(df)
        print(f"Loaded {num_events} events successfully into Snowflake: {table_name}.")
    except Exception as e:
        print(f"Failed to load data into Snowflake: {e}")
    finally:
        # Close connection
        engine.dispose()
        

In [13]:

#load_snowflake(df_clean, "Events")

In [14]:
# for col in df_clean.columns:
#     if any(isinstance(i, (dict, list)) for i in df_clean[col]):
#         print(f"Column '{col}' contains nested structures.")

In [15]:
df_clean.to_csv("test.csv")

In [16]:
print(df_clean1.columns)

Index(['name', 'type', 'id', 'url', 'classifications', 'info', 'pleaseNote',
       'priceRanges', 'sales_public_startDateTime', 'sales_public_startTBD',
       'sales_public_startTBA', 'sales_public_endDateTime',
       'dates_access_startDateTime', 'dates_access_startApproximate',
       'dates_access_endDateTime', 'dates_access_endApproximate',
       'dates_start_localDate', 'dates_start_localTime',
       'dates_start_dateTime', 'dates_start_dateTBD', 'dates_start_dateTBA',
       'dates_start_timeTBA', 'dates_start_noSpecificTime',
       'dates_end_localDate', 'dates_end_localTime', 'dates_end_dateTime',
       'dates_end_approximate', 'dates_end_noSpecificTime', 'dates_timezone',
       'dates_status_code', 'dates_spanMultipleDays', '_links_self_href',
       '_links_venues', '_embedded_venues', '_links_attractions',
       '_embedded_attractions', 'ticketing_safeTix_enabled',
       'ticketing_safeTix_inAppOnlyEnabled', 'ticketing_id', 'description',
       'promoters', 'promo